In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from IPython.display import Image


In [4]:
import torch
import numpy as np
import os
import torch.nn as nn
from tqdm import tqdm
import json
from functools import partial
from torch import einsum, nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from einops import pack, rearrange, reduce, repeat, unpack


In [5]:
def findAllFile(base):
    file_path = []
    for root, ds, fs in os.walk(base, followlinks=True):
        for f in fs:
            fullname = os.path.join(root, f)
            file_path.append(fullname)
    return file_path



## Helpers

In [7]:
from configs.config_vqvae import cfg, get_cfg_defaults
def load_vqvae(gen_cfg):

    body_cfg = get_cfg_defaults()
    body_cfg.merge_from_file(gen_cfg.vqvae.body_config)
    body_model = (
        instantiate_from_config(body_cfg.vqvae).to(device).eval()
    )
    body_model.load(os.path.join(body_cfg.output_dir, "vqvae_motion.pt"))

    if gen_cfg.vqvae.left_hand_config is  None and gen_cfg.vqvae.right_hand_config is None:
        return body_model, body_cfg
    
    if gen_cfg.vqvae.left_hand_config is not None:
        left_cfg = get_cfg_defaults()
        left_cfg.merge_from_file(gen_cfg.vqvae.left_hand_config)
        left_hand_model = instantiate_from_config(left_cfg.vqvae).to(device).eval()
        left_hand_model.load(
            os.path.join(left_cfg.output_dir, "vqvae_motion.pt")
        )
    else:
        left_hand_model = None
        
    if gen_cfg.vqvae.right_hand_config is not None:
        right_cfg = get_cfg_defaults()
        right_cfg.merge_from_file(gen_cfg.vqvae.right_hand_config)
        right_hand_model = instantiate_from_config(right_cfg.vqvae).to(device).eval()
        right_hand_model.load(
            os.path.join(right_cfg.output_dir, "vqvae_motion.pt")
        )
    else:
        right_hand_model = None

    return body_model, left_hand_model , right_hand_model , body_cfg , left_cfg , right_cfg


In [32]:
@torch.no_grad()
def bkn_to_motion( codes, dset , remove_translation = True):
    # codes b k n

    k = codes.shape[1]
    mrep = dset.motion_rep

    if k == 1:
        if mrep == MotionRep("body"):

            body_inds = codes[:, 0]
            body_motion = body_model.decode(body_inds[0:1]).detach().cpu()

            if remove_translation:
                z = torch.zeros(
                    body_motion.shape[:-1] + (2,),
                    dtype=body_motion.dtype,
                    device=body_motion.device,
                )
                body_motion = torch.cat(
                    [body_motion[..., 0:1], z, body_motion[..., 1:]], -1
                )

            body_M = dset.toMotion(
                body_motion[0],
                motion_rep=MotionRep("body"),
                hml_rep=body_cfg.dataset.hml_rep,
            )

            return body_M

        elif mrep == MotionRep("left_hand"):

            left_inds = codes[:, 0]
            left_motion = left_hand_model.decode(left_inds[0:1]).detach().cpu()
            left_M = dset.toMotion(
                left_motion[0],
                motion_rep=MotionRep(left_cfg.dataset.motion_rep),
                hml_rep=left_cfg.dataset.hml_rep,
            )
            return left_M

        elif mrep == MotionRep("right_hand"):
            right_inds = codes[:, 0]
            right_motion = (
                right_hand_model.decode(right_inds[0:1]).detach().cpu()
            )
            right_M = dset.toMotion(
                right_motion[0],
                motion_rep=MotionRep(right_cfg.dataset.motion_rep),
                hml_rep=right_cfg.dataset.hml_rep,
            )
            return right_M

    if k == 2:
        left_inds = codes[:, 0]
        right_inds = codes[:, 1]

        left_motion = left_hand_model.decode(left_inds[0:1]).detach().cpu()
        right_motion = right_hand_model.decode(right_inds[0:1]).detach().cpu()

        left_M = dset.toMotion(
            left_motion[0],
            motion_rep=MotionRep(left_cfg.dataset.motion_rep),
            hml_rep=left_cfg.dataset.hml_rep,
        )
        right_M = dset.toMotion(
            right_motion[0],
            motion_rep=MotionRep(right_cfg.dataset.motion_rep),
            hml_rep=right_cfg.dataset.hml_rep,
        )
        hand_M = left_M + right_M
        hand_M.motion_rep = MotionRep.HAND
        hand_M.hml_rep = "".join(
            [i for i in left_M.hml_rep if i in right_M.hml_rep]
        )
        return hand_M

    if k == 3:
        left_inds = codes[:, 1]
        right_inds = codes[:, 2]
        body_inds = codes[:, 0]
        body_motion = body_model.decode(body_inds[0:1]).detach().cpu()

        
        if remove_translation:
            z = torch.zeros(
                body_motion.shape[:-1] + (2,),
                dtype=body_motion.dtype,
                device=body_motion.device,
            )
            body_motion = torch.cat([body_motion[..., 0:1], z, body_motion[..., 1:]], -1)

        left_motion = left_hand_model.decode(left_inds[0:1]).detach().cpu()
        right_motion = right_hand_model.decode(right_inds[0:1]).detach().cpu()

        body_M = dset.toMotion(
            body_motion[0],
            motion_rep=MotionRep("body"),
            hml_rep = body_cfg.dataset.hml_rep)

        left_M = dset.toMotion(
            left_motion[0],
            motion_rep=MotionRep("left_hand"),
            hml_rep=left_cfg.dataset.hml_rep)
        right_M = dset.toMotion(
            right_motion[0],
            motion_rep=MotionRep("right_hand"),
            hml_rep=right_cfg.dataset.hml_rep)
        full_M = dset.to_full_joint_representation(body_M, left_M, right_M)
        return full_M



In [11]:
from core.models.utils import instantiate_from_config, get_obj_from_str


In [43]:
body_cfg_gprvc = get_cfg_defaults()
body_cfg_gprvc.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/vqvae/vqvae_full_gpvc/vqvae_full_gpvc.yaml")
full_gpvc = instantiate_from_config(body_cfg_gprvc.vqvae).to(device).eval()
full_gpvc.load("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/vqvae/vqvae_full_gpvc/vqvae_motion.pt")


In [9]:
from core import MotionTokenizerParams, pattern_providers

from core.param_dataclasses import pattern_providers
from core.datasets.multimodal_dataset import MotionIndicesAudioTextDataset, load_dataset_gen, simple_collate
from core.models.utils import instantiate_from_config, get_obj_from_str
from core import MotionRep, AudioRep, TextRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser
from core.models.generation.muse import MLMModel, MotionMuse
import einops
from configs.config_t2m import get_cfg_defaults as muse_get_cfg_defaults
from core import MotionTokenizerParams

from core.models.generation.translation_transformer import TranslationTransformer
from core.datasets.text_encoders import BERTConditioner, ClipConditioner, T5Conditioner, parse_prompt_attention
from core.datasets.audio_encoders import EncodecConditioner, LibrosaConditioner

/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
gen_cfg = muse_get_cfg_defaults()
gen_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_muse_critic/motion_muse_critic.yaml")
gen_cfg.freeze()
tranformer_config = gen_cfg.motion_generator
fuse_config = gen_cfg.fuser
pattern_config = gen_cfg.codebooks_pattern
dataset_args = gen_cfg.dataset

target = tranformer_config.pop("target")


In [11]:
motion_gen = MotionMuse(tranformer_config , fuse_config , pattern_config).to(device)
# motion_gen = torch.compile(motion_gen)
pkg = torch.load("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_muse_critic/motion_muse.pt", map_location="cuda")
motion_gen.load_state_dict(pkg["model"])

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


<All keys matched successfully>

In [12]:
pkg["steps"]

tensor([110000.], device='cuda:0')

In [108]:
body_model, body_cfg = load_vqvae(gen_cfg)

Sync is turned on False
loaded model with  0.03015906736254692 tensor([110000.], device='cuda:0') steps


In [109]:
gen_cfg2 = muse_get_cfg_defaults()
gen_cfg2.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_muse_hands/motion_muse_hands.yaml")
gen_cfg2.freeze()
tranformer_config2 = gen_cfg2.motion_generator
fuse_config2 = gen_cfg2.fuser
pattern_config2 = gen_cfg2.codebooks_pattern
dataset_args2 = gen_cfg2.dataset

_ = tranformer_config2.pop("target")


In [110]:
motion_gen2 = MotionMuse(tranformer_config2 , fuse_config2 , pattern_config2).to(device)
# motion_gen = torch.compile(motion_gen)
pkg2 = torch.load("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_muse_hands/motion_muse.pt", map_location="cuda")
motion_gen2.load_state_dict(pkg2["model"])

<All keys matched successfully>

In [197]:
# from core.models.generation.translation_transformer import TranslationTransformer
# from configs.config_t2m import get_cfg_defaults as trans_get_cfg_defaults

# t_cfg = trans_get_cfg_defaults()
# t_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_translation/motion_translation.yaml")
# # t_cfg.freeze()

# tranformer_config2 = t_cfg.translation_transformer
# fuse_config2 = t_cfg.fuser
# dataset_args2 = t_cfg.dataset

# target2 = tranformer_config2.pop("target")

# trans_former = TranslationTransformer(tranformer_config2, fuse_config2).to(device)
# trans_former.load("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_translation/translation_transformer.pt")

In [111]:
_, left_hand_model , right_hand_model , _ , left_cfg , right_cfg = load_vqvae(gen_cfg2)

In [112]:
condition_provider = ConditionProvider(
            text_conditioner_name = dataset_args.text_conditioner_name,
            motion_rep=MotionRep(dataset_args.motion_rep),
            audio_rep=AudioRep(dataset_args.audio_rep),
            text_rep=TextRep(dataset_args.text_rep),
            motion_padding=dataset_args.motion_padding,
            audio_padding=dataset_args.audio_padding,
            motion_max_length_s=dataset_args.motion_max_length_s,
            audio_max_length_s=dataset_args.audio_max_length_s,
            pad_id = MotionTokenizerParams(tranformer_config.num_tokens).pad_token_id,
            fps=30/4,
            # device = "cpu"
        )

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [113]:
from core.datasets.base_dataset import BaseMotionDataset
base_dset = BaseMotionDataset(motion_rep=MotionRep.BODY , hml_rep= "gpvc")

In [114]:
beat = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/beat/1_wayne_0_1_1.wav"
choreo = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/choreomaster/0071.wav"
mbr = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/aist/mBR0.wav"
mj = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/aist/mJB0.wav"
wild =  "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/wild/despacito.mp3"

In [155]:
_, conditions = condition_provider(raw_audio= None, raw_text="A person walks forwards and then sits." )
_, neg_conditions = condition_provider(raw_text="body movement")
gen_ids = motion_gen.generate(conditions =conditions, neg_conditions = None, duration_s = 4, temperature = 0.2 ,timesteps=24, cond_scale = 3)


100%|███████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 33.00it/s]


In [156]:
gen_ids2 = motion_gen2.generate(conditions =conditions, neg_conditions = None, duration_s = 4, temperature = 0.2 ,timesteps=24, cond_scale = 3)


100%|███████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 49.50it/s]


In [157]:
all_gen_ids = torch.cat([gen_ids , gen_ids2] , 1)

In [158]:
gen_motion = bkn_to_motion(all_gen_ids, base_dset)
base_dset.render_hml(
                    gen_motion,
                    f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/gen_novel_full.gif",
                    zero_trans = True,
                    zero_orient = True,
    
                )

In [13]:
# Image(open(f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/gen_novel_full.gif",'rb').read())

In [49]:
_, conditions = condition_provider(raw_audio= beat, raw_text="a person gives a speech angrily with exagerrated hand movements"  , audio_max_length_s=16)
_, neg_conditions = condition_provider(raw_text="dancing")

In [19]:
conditions["audio"][0].shape

torch.Size([1, 480, 128])

In [40]:
_, conditions = condition_provider(raw_audio= None , raw_text = "a person jumps once and turns around")
_, neg_conditions = condition_provider(raw_text="jittery motion, slow, stop")

In [41]:
conditions["audio"][0].shape

torch.Size([1, 1, 128])

In [42]:
raw_audio= None
raw_text = "a person jumps once and turns around"
neg_text = None
neg_audio = None
critic = True
nme = "" if raw_audio is None else os.path.basename(raw_audio).split('.')[0]
nme += f"{raw_text}_{critic}"
nme += "_neg_"
nme = nme + ("" if neg_audio is None else os.path.basename(neg_audio).split('.')[0])
nme = nme + ("" if neg_text is None else f"{neg_text}")

In [43]:
nme

'a person jumps once and turns around_True_neg_'

In [44]:
_, conditions = condition_provider(raw_audio= raw_audio, raw_text= raw_text   )
_, neg_conditions = condition_provider(raw_text=neg_text)


In [50]:
gen_ids = motion_gen.generate(conditions =conditions, neg_conditions = None, \
                              duration_s = 4, temperature = 1.0 ,timesteps=24, \
                              cond_scale = 8,force_not_use_token_critic = critic)



100%|███████████████████████████████████████████████████████████████████| 72/72 [00:03<00:00, 23.27it/s]


motion representation: full, type: gpvc, num joints: 52, dim: 317


In [51]:
gen_motion = bkn_to_motion(gen_ids, base_dset)
base_dset.render_hml(
                    gen_motion,
                    f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/gen_movel.gif",
                    zero_trans = True,
                    zero_orient = True,
    
                )

In [22]:
# Image(open(f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/{nme}.gif",'rb').read())

## Generate aniamtion

In [13]:
from core.models.generation.muse import generate_animation

In [15]:
from core import MotionTokenizerParams
from core.models.utils import instantiate_from_config, get_obj_from_str
from core import MotionRep, AudioRep, TextRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser
from core.models.generation.muse import MotionMuse

import einops
from configs.config_t2m import get_cfg_defaults as muse_get_cfg_defaults
from core import MotionTokenizerParams

In [16]:
gen_cfg = muse_get_cfg_defaults()
gen_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/MMCMG/checkpoints/motion_muse_body_hands/motion_muse_body_hands.yaml")
gen_cfg.freeze()
tranformer_config = gen_cfg.motion_generator
fuse_config = gen_cfg.fuser
dataset_args = gen_cfg.dataset

target = tranformer_config.pop("target")


In [17]:
motion_gen = MotionMuse(tranformer_config , fuse_config).to(device).eval()
pkg = torch.load("/srv/hays-lab/scratch/sanisetty3/music_motion/MMCMG/checkpoints/motion_muse_body_hands/motion_muse.pt", map_location="cuda")
motion_gen.load_state_dict(pkg["model"])
motion_gen = torch.compile(motion_gen)


Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


Process ForkProcess-3:
Process ForkProcess-6:
Process ForkProcess-2:
Process ForkProcess-4:
Process ForkProcess-5:
Process ForkProcess-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/multiprocessing/process.py", line 3

In [17]:
# body_model , body_cfg= load_vqvae(gen_cfg)

In [20]:
body_model, left_hand_model , right_hand_model , body_cfg , left_cfg , right_cfg = load_vqvae(gen_cfg)

Sync is turned on False
loaded model with  0.03015906736254692 tensor([110000.], device='cuda:0') steps


In [21]:
condition_provider = ConditionProvider(
            text_conditioner_name = dataset_args.text_conditioner_name,
            motion_rep=MotionRep(dataset_args.motion_rep),
            audio_rep=AudioRep(dataset_args.audio_rep),
            text_rep=TextRep(dataset_args.text_rep),
            motion_padding=dataset_args.motion_padding,
            audio_padding=dataset_args.audio_padding,
            motion_max_length_s=dataset_args.motion_max_length_s,
            audio_max_length_s=dataset_args.audio_max_length_s,
            pad_id = MotionTokenizerParams(tranformer_config.num_tokens).pad_token_id,
            fps=30/4,
        )

config.json: 100%|█████████████████████████████████████████████████████| 809/809 [00:00<00:00, 329kB/s]
model.safetensors: 100%|██████████████████████████████████████████| 93.1M/93.1M [00:02<00:00, 45.5MB/s]
/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
preprocessor_config.json: 100%|████████████████████████████████████████| 234/234 [00:00<00:00, 100kB/s]
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [26]:
from core.datasets.base_dataset import BaseMotionDataset
base_dset = BaseMotionDataset(motion_rep=MotionRep.FULL , hml_rep= "gpvc")

In [40]:
from visualization.joints2bvh import Joint2BVHConvertor
converter = Joint2BVHConvertor(mode = "smplx")

In [27]:
beat = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/beat/1_wayne_0_1_1.wav"
choreo = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/choreomaster/0071.wav"
mbr = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/aist/mBR0.wav"
mj = "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/aist/mJB0.wav"
wild =  "/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/wild/despacito.mp3"

In [28]:
aud_clip =  wild
text_ = "A person dancing energetically"
neg_text = None
use_token_critic = True
temperature = 0.6

sve_file = '|'.join(t for t in text_) if isinstance(text_ , list) else f"{text_}"
sve_file += "" if aud_clip is None else f"{os.path.basename(aud_clip).split('.')[0]}"
sve_file += f"_use_critic_{use_token_critic}_{temperature}"


In [29]:
print(sve_file)

A person dancing energeticallydespacito_use_critic_True_0.6


In [30]:
all_ids_body = generate_animation(motion_gen , condition_provider ,overlap = 10, duration_s = 30 , temperature = temperature, aud_file=aud_clip, text = text_ , neg_text=neg_text,use_token_critic = use_token_critic, timesteps = 24 )

100%|██████████████████████████████████████████████████████████████████| 72/72 [00:01<00:00, 47.02it/s]


In [33]:
gen_motion = bkn_to_motion(all_ids_body, base_dset)

### Save as gif

In [46]:
base_dset.render_hml(
                    gen_motion,
                    f"/srv/hays-lab/scratch/sanisetty3/music_motion/MMCMG/renders/{sve_file}.gif",
                    zero_trans = True,
                    zero_orient = True,
    
                )

### Save as BVH

In [42]:
motion_inv = base_dset.inv_transform(gen_motion)
motion_xyz = base_dset.to_xyz(motion_inv)

In [45]:
_, ik_joint = converter.convert(motion_xyz.cpu().numpy(), filename=f"/srv/hays-lab/scratch/sanisetty3/music_motion/MMCMG/renders/{sve_file}.bvh", iterations=10, foot_ik = False)

In [54]:
import moviepy.editor as mp
import math

def joinGifAud(vid , aud_clip , sve_name = "vid"):
    video_clip = mp.VideoFileClip(vid)
    audio_clip = mp.AudioFileClip(aud_clip)
    endd = int(math.floor(min(audio_clip.end , video_clip.end)))
    video_clip = video_clip.subclip(0, endd)
    final_clip = video_clip.set_audio(audio_clip.subclip(0, endd))
    save_name = os.path.basename(aud_clip).split('.')[0]
    final_clip.write_videofile(f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/{sve_name}.mp4")


In [62]:
aud_clip

'/srv/hays-lab/scratch/sanisetty3/motionx/audio/wav/beat/1_wayne_0_1_1.wav'

In [63]:
joinGifAud(f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/2024/{sve_file}.gif" , aud_clip , sve_name = f"{sve_file}_full")

Moviepy - Building video /srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/1_wayne_0_1_1_use_critic_True_0.4_full.mp4.
MoviePy - Writing audio in 1_wayne_0_1_1_use_critic_True_0.4_fullTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/1_wayne_0_1_1_use_critic_True_0.4_full.mp4



Moviepy - Done !
Moviepy - video ready /srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/1_wayne_0_1_1_use_critic_True_0.4_full.mp4
